In [ ]:
# 解压数据,数据来源：https://mosmed.ai/
#原本数据是NiFit格式数据，然后转换成png格式，输入PaddleSeg进行训练
!unzip data/data114821/MosMedSegPNG.zip -d /home/aistudio/work

In [ ]:
#支持配置化训练和API方式训练,这里采用API方式进行训练
!pip install paddleseg SimpleITK

In [ ]:
#加载常用库
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import cv2
import paddle
import paddleseg

In [ ]:
### 划分数据集
random.seed(1000)
path_origin = '/home/aistudio/work/MosMedSegPNG/origin'
files = list(filter(lambda x: x.endswith('.png'), os.listdir(path_origin)))
random.shuffle(files)
rate = int(len(files) * 0.8)#训练集和测试集8：2
train_txt = open('/home/aistudio/work/MosMedSegPNG/train_list.txt','w')
val_txt = open('/home/aistudio/work/MosMedSegPNG/val_list.txt','w')
for i,f in enumerate(files):
    image_path = os.path.join(path_origin, f)
    label_path = image_path.replace("origin", "mask")
    if i < rate:
        train_txt.write(image_path + ' ' + label_path+ '\n')
    else:
        val_txt.write(image_path + ' ' + label_path+ '\n')
train_txt.close()
val_txt.close()

print('完成')

In [ ]:
import paddleseg.transforms as T
from paddleseg.datasets import Dataset

train_transforms = [
    T.Resize(target_size=(550, 550)),
    T.RandomHorizontalFlip(),
    T.RandomDistort(brightness_range = 0.2,
                 contrast_range = 0.2,
                 saturation_range = 0.2,
                 hue_prob = 0),
    T.RandomRotation(max_rotation = 10,im_padding_value =(0,0,0),label_padding_value = 0),#随机旋转
    T.RandomBlur(),
    T.RandomPaddingCrop(crop_size = (512, 512),
                 im_padding_value = (0,0,0),
                 label_padding_value = 0),
    T.Normalize()
]
val_transforms = [
    T.Resize(target_size=(512, 512)),
    T.Normalize()
]

dataset_root = '/home/aistudio/work/MosMedSegPNG'
train_path  = '/home/aistudio/work/MosMedSegPNG/train_list.txt'
val_path  = '/home/aistudio/work/MosMedSegPNG/val_list.txt'
# 构建训练集
train_dataset = Dataset(
    transforms = train_transforms,
                  dataset_root = dataset_root,
                  num_classes = 2,
                  train_path  = train_path,
                  mode = 'train'
                  )
#验证集
val_dataset = Dataset(
    transforms = val_transforms,
                  dataset_root = dataset_root,
                  num_classes = 2,
                  val_path = val_path,
                  mode = 'val'
                  )

In [ ]:
import numpy as np
import cv2
import os
 
# img_h, img_w = 32, 32
img_h, img_w = 512, 512   #根据自己数据集适当调整，影响不大
means, stdevs = [], []
img_list = []
 
imgs_path = '/home/aistudio/work/MosMedSegPNG/origin'
imgs_path_list = os.listdir(imgs_path)
 
len_ = len(imgs_path_list)
i = 0
for item in imgs_path_list:
    img = cv2.imread(os.path.join(imgs_path,item))
    try:
        img = cv2.resize(img,(img_w,img_h))
    except:
        continue
    img = img[:, :, :, np.newaxis]
    img_list.append(img)
    i += 1
    print(i,'/',len_)    
 
imgs = np.concatenate(img_list, axis=3)
imgs = imgs.astype(np.float32) / 255.
 
for i in range(3):
    pixels = imgs[:, :, i, :].ravel()  # 拉成一行
    means.append(np.mean(pixels))
    stdevs.append(np.std(pixels))
 
# BGR --> RGB ， CV读取的需要转换，PIL读取的不用转换
means.reverse()
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))

In [ ]:
# 预览经过数据增强后的数据。
plt.figure(figsize=(16,16))
for i in range(1,6,2):
    img, label = train_dataset[50]
    img = np.transpose(img, (1,2,0))
    img = img*0.5 + 0.5
    plt.subplot(3,2,i),plt.imshow(img,'gray'),plt.title('img'),plt.xticks([]),plt.yticks([])
    plt.subplot(3,2,i+1),plt.imshow(label,'gray'),plt.title('label'),plt.xticks([]),plt.yticks([])
    plt.show

In [ ]:
from paddleseg.models import UNet, BiSeNetV2
from paddleseg.core import train
from paddleseg.models.losses import CrossEntropyLoss,DiceLoss, MixedLoss
import paddle
num_classes = 2
model = BiSeNetV2(num_classes=num_classes)

# 设置学习率  
batch_size=8
log_iters = int(len(train_dataset)/batch_size /3) # 日志打印间隔
iters = int(len(train_dataset)/batch_size) * 200 # 训练次数
save_interval = int(len(train_dataset)/batch_size) * 5 # 保存的间隔次数
base_lr = 0.02
# 优化器和损失函数
lr = paddle.optimizer.lr.PolynomialDecay(base_lr, power=0.9, decay_steps=iters, end_lr=0.0000125)
optimizer = paddle.optimizer.Momentum(lr, parameters=model.parameters(), momentum=0.9, weight_decay=4.0e-5)
mixtureLosses = [CrossEntropyLoss(),DiceLoss() ]
mixtureCoef = [0.7,0.3]
losses = {}
losses['types'] = [MixedLoss(mixtureLosses, mixtureCoef)]*5
losses['coef'] = [1]*5

train(
    model=model,
    train_dataset=train_dataset,# 填写训练集的dataset
    val_dataset=val_dataset,# 填写验证集的dataset
    optimizer=optimizer,# 优化器
    save_dir='/home/aistudio/Bisnet',# 保存路径
    iters=iters,
    batch_size=batch_size,
    save_interval=save_interval,
    log_iters=log_iters,
    losses=losses,
    use_vdl=True)# 是否使用visualDL

In [ ]:
import paddle
from paddleseg.core import evaluate
from paddleseg.models import BiSeNetV2
model = BiSeNetV2(num_classes=2)
# 训练的模型保存结果路径
model_path = '/home/aistudio/MyBestModel/BiSeNet_model.pdparams'
para_state_dict = paddle.load(model_path)
model.set_dict(para_state_dict)
evaluate(model,val_dataset)

In [ ]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import paddle
import paddleseg.transforms as T
from paddleseg.core import infer
from paddleseg.models import UNet,BiSeNetV2

def nn_infer(model, im, model_path):
    # 网络定义
    para_state_dict = paddle.load(model_path)
    model.set_dict(para_state_dict)
    # 预测结果
    transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.Normalize()
        ])
    img, _ = transforms(im)
    img = paddle.to_tensor(img[np.newaxis, :])
    pre = infer.inference(model, img)
    pred = paddle.argmax(pre, axis=1).numpy().reshape((512, 512))
    return pred.astype('uint8')

params = '/home/aistudio/MyBestModel/BiSeNet_model.pdparams'
model = BiSeNetV2(num_classes=2 )
img_path = '/home/aistudio/work/MosMedSegPNG/origin/study_0271_20.png'
lab_path = '/home/aistudio/work/MosMedSegPNG/mask/study_0271_20.png'

img = np.asarray(Image.open(img_path))
lab = np.asarray(Image.open(lab_path))
# 还原大小
pre = cv2.resize(nn_infer(model, img, params), (lab.shape[1], lab.shape[0]), cv2.INTER_NEAREST)
plt.figure(figsize=(15, 10))
plt.subplot(131);plt.imshow(img,'gray');plt.title('image')
plt.subplot(132);plt.imshow(lab,'gray');plt.title('label')
plt.subplot(133);plt.imshow(pre,'gray');plt.title('pre')
plt.show()

In [ ]:
import SimpleITK as sitk
from paddleseg.core import infer
def wwwc(sitkImage,ww=1500,wc=-550):
    # 设置窗宽窗位
    min = int(wc - ww/2.0)
    max = int(wc + ww/2.0)
    intensityWindow = sitk.IntensityWindowingImageFilter()
    intensityWindow.SetWindowMaximum(max)
    intensityWindow.SetWindowMinimum(min)
    sitkImage = intensityWindow.Execute(sitkImage)
    return sitkImage

def readNii(path,isflipud=True):
    """读取和加载数据"""
    img = wwwc(sitk.ReadImage(path))
    data = sitk.GetArrayFromImage(img)
    # 图像是上下翻转的，所有把他们翻转过来
    if isflipud:
        data = np.flip(data,1)
    return data

def nn_infer(model, im):
    # 预测结果
    transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.Normalize()
        ])
    img, _ = transforms(im)
    img = paddle.to_tensor(img[np.newaxis, :])
    pre = infer.inference(model, img)
    pred = paddle.argmax(pre, axis=1).numpy().reshape((512, 512))
    return pred.astype('uint8')

In [ ]:
# 原始医疗数据的文件路径，格式Nifit
origin_f_path = '/home/aistudio/study_0306.nii'
# 通过SimpleITK读取，并设置窗宽窗位并缩放到0~255之间，并转换成numpy格式
origin_numpy = readNii(origin_f_path).astype(np.uint8)
d,h,w = origin_numpy.shape
# 用来保存结果。
result = np.zeros((d,h,w,3)).astype(np.uint8)
mask_numpy = np.zeros_like(origin_numpy).astype(np.uint8)

model_path = '/home/aistudio/MyBestModel/BiSeNet_model.pdparams'
model = BiSeNetV2(num_classes=2 )
para_state_dict = paddle.load(model_path)
model.set_dict(para_state_dict)

for i in range(d):
    img = origin_numpy[i].copy()
    img = np.expand_dims(img, axis=2)
    img = np.concatenate((img, img, img), axis=-1).astype(np.uint8)
    pre = cv2.resize(nn_infer(model, origin_numpy[i]), (512,512), cv2.INTER_NEAREST)
    mask_numpy[i] = pre
    ret,thresh = cv2.threshold(pre,0,255,cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, kernel=np.ones((5, 5), np.uint8), iterations=1)
    contours, hierarchy = cv2.findContours(thresh, 1, 2)
    # 这是画轮廓
    img = cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    result[i] =  img

# 把预测出来的mask保存成三维的Nifit格式。
mask_numpy = np.flip(mask_numpy, 1)
pre_sitkImage = sitk.GetImageFromArray(mask_numpy)
pre_sitkImage.CopyInformation(sitk.ReadImage(origin_f_path))
pre_sitkImage = sitk.Cast(pre_sitkImage, sitk.sitkUInt8)
save_path =origin_f_path.split('.')[0] + '_mask.nii'
sitk.WriteImage(pre_sitkImage, save_path)

plt.figure(figsize=(15, 15))
plt.subplot(221);plt.imshow(result[20])
plt.subplot(222);plt.imshow(result[22])
plt.subplot(223);plt.imshow(result[24])
plt.subplot(224);plt.imshow(result[26])
plt.show()